In [1]:
import json

In [2]:
def find_substring_indices(parent_string, substring):  
    start_index = parent_string.find(substring)  
    if start_index != -1:
        end_index = start_index + len(substring)
        return start_index, end_index  
    else:  
        return -1, -1

In [3]:
parent_string = "三大攻坚战取得关键进展"  
substring = "三大攻坚战"  
start_idx, end_idx = find_substring_indices(parent_string, substring)  
  
if start_idx != -1 and end_idx != -1:  
    print(f"子字符串 '{substring}' 在父字符串中的开始下标是 {start_idx}，结束下标是 {end_idx}")  
else:  
    print(f"子字符串 '{substring}' 没有在父字符串中找到")

子字符串 '三大攻坚战' 在父字符串中的开始下标是 0，结束下标是 5


In [4]:
from pprint import pprint

In [5]:
# {"id":17168,"text":"三大攻坚战取得关键进展","label":[[0,5,"任务"]],"Comments":[]}

In [6]:
len("三大攻坚战取得关键进展")

11

In [7]:
import re

def tran_llm_doccano(input_file, output_file, schema):
    doccano_format = {
        "text": None,
        "label": [],
        "Comments": []
    }

    def _find_text(text):
        pattern = r'text:(.*?)",'  
        match = re.search(pattern, text, re.MULTILINE)
        text_content = match.group(1)
        return text_content

    with open(input_file, 'r') as f:
        with open(output_file, 'w') as w:
            for line in f:
                text = _find_text(line)
                doccano_format["text"] = text
                data = json.loads(line)
                predict = data["predict"]
                tmp = []
                for ent_cls in schema:
                    for predict_ent_name in predict[ent_cls]:
                        start_idx, end_idx = find_substring_indices(text, predict_ent_name)
                        if start_idx == -1 or end_idx == -1:
                            continue
                        tmp.append([start_idx, end_idx, ent_cls])
                doccano_format["label"] = tmp
            w.write(json.dumps(doccano_format, ensure_ascii=False) + '\n')

schema = ['数据', '项目', '任务']
tran_llm_doccano('llm_predict.jsonl', "doccano_import.jsonl", schema)